In [1]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_predict, GridSearchCV, cross_val_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics

In [2]:
def setpvalue(series):
    return (series<0.01).replace({True:'*',False:''}) + (series<0.05).replace({True:'*',False:''}) + (series<0.1).replace({True:'*',False:''})

In [3]:
raw = pd.read_csv('dataset'+str(1040)+'.csv') 

In [ ]:
raw.columns[7:15]

,c_birthyear,c_death_age,c_fl_earliest_year,c_fl_latest_year,Longitude,Latitude,status,nemesis
0,0,0,1074,1074,0.000000,0.000000,0.000000,0.000000
1,0,0,0,0,112.382630,34.665276,0.000000,0.000000
2,0,0,0,1043,0.000000,0.000000,0.000000,0.000000
3,0,0,0,0,116.351341,27.984781,0.000000,0.000000
4,0,0,0,0,120.578262,30.004515,0.000000,0.000000
5,998,44,0,0,114.343330,34.785477,0.000000,0.000000
6,0,76,0,0,119.321577,26.073954,0.000000,0.000000
7,985,66,0,0,120.618622,31.312710,0.000000,0.000000
8,0,0,0,0,114.343330,34.785477,0.000000,0.000000
9,0,0,0,0,114.856377,34.063873,0.000000,0.000000


In [18]:
raw.iloc[:,45:137]

,choronym_1,choronym_2,choronym_3,choronym_4,choronym_6,choronym_7,choronym_8,choronym_9,choronym_10,choronym_11,...,choronym_106,choronym_108,choronym_109,choronym_110,choronym_112,choronym_115,choronym_117,choronym_147,choronym_154,choronym_156
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
raw.iloc[:,137:-4]

,entry_0,entry_1,entry_5,entry_7,entry_8,entry_14,entry_15,entry_21,entry_26,entry_27,...,entry_145,entry_161,entry_163,entry_165,entry_166,entry_168,entry_169,entry_225,entry_226,entry_248
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Linear Regression

In [4]:
import scipy.stats as sp # for spearman correlation
import statsmodels.api as sm
from scipy import stats # for logistic regression
from statsmodels.formula.api import ols

In [44]:
def analysis(year,spearmanflag):   
   
    # Read raw file
    raw = pd.read_csv('dataset'+str(year)+'.csv') 
    
    # Fill missing value: years, ages with mean
    for columnname in raw.iloc[:,7:15].columns:
        meanvalue = raw[raw[columnname] !=0][columnname].mean()
        raw[columnname] = raw[columnname].replace({0:meanvalue})
    
    
    dataset_X = pd.concat([raw.iloc[:,7],raw.iloc[:,9:11], raw.iloc[:,12:15]#,raw.iloc[:,-2:]
                          ], axis = 1)
    dataset_y = raw.iloc[:,2]
    
    glm = sm.OLS(dataset_y, dataset_X)
    resultmodel = glm.fit()
    
    coeffandp = pd.concat([pd.DataFrame(resultmodel.params, columns = [year]),\
                           pd.DataFrame(resultmodel.pvalues, columns = ['pvalue'])] , axis  = 1)
    
    
    
    coeffandp['sx'] = np.where(coeffandp.pvalue < 0.10, '*', '')
    coeffandp['s5'] = np.where(coeffandp.pvalue < 0.05, '*', '')
    coeffandp['s1'] = np.where(coeffandp.pvalue < 0.01, '*', '')
    coeffandp[str(year)+'mk'] = coeffandp['sx'] + coeffandp['s5'] + coeffandp['s1']
    
    coeffandp.drop(['sx','s5','s1','pvalue'], inplace = True, axis =1)
    
    if (spearmanflag == 1):
        
        fulldata = pd.concat([dataset_y,dataset_X], axis = 1)

        spearmanresult = sp.spearmanr(fulldata)

        spearmandf = pd.DataFrame(spearmanresult.correlation, index = fulldata.columns)
        spearmandf.columns = fulldata.columns
        spearmandfp = pd.DataFrame(spearmanresult.pvalue, index = fulldata.columns)
        spearmandfp.columns = fulldata.columns
                                      
        return coeffandp, resultmodel, spearmandf, spearmandfp
    else:
        return coeffandp, resultmodel
                                      
                                

        
    

In [45]:
coefdict = {}
modeldict = {}
spearmandict = {}

for startingyear in range(960,1280,10):

    coeffdf, model, spearmandf, spearmandfp = analysis(startingyear,1)
    
    coefdict[startingyear] = coeffdf
    modeldict[startingyear] = model
    spearmandict[startingyear] = (spearmandf,spearmandfp)

### R-square

In [46]:
dflist = []

for year in range(960,1280,10):
    stats ={}
    stats['R-squared'] = modeldict[year].rsquared
    stats['Adjusted R-squared'] = modeldict[year].rsquared_adj
    stats['F-statistic'] = modeldict[year].fvalue
    stats['F-stat p-value'] = modeldict[year].f_pvalue
    
    dflist.append(pd.DataFrame(stats,index =[year]))

In [47]:
# R-squared table

significantvariablenumber = {}
for startingyear in range(960,1280,10):
    significantvariablenumber[startingyear] = len(coefdict[startingyear][coefdict[startingyear].iloc[:,-1] != ''])
    
    
rsquare = pd.concat(dflist)
rsquare.iloc[:,-1] = setpvalue(rsquare.iloc[:,-1])
pd.concat([rsquare,pd.DataFrame(significantvariablenumber, index = ['Number of Variable found significant']).T],axis =1)#.to_csv('newrsquare.csv')

,R-squared,Adjusted R-squared,F-statistic,F-stat p-value,Number of Variable found significant
960,0.828225,0.819416,94.020360,***,2
970,0.843949,0.838376,151.428302,***,2
980,0.856935,0.852664,200.659264,***,1
990,0.860920,0.856928,215.622652,***,1
1000,0.848984,0.845628,252.982060,***,1
1010,0.872638,0.870197,357.426827,***,2
1020,0.863491,0.860832,324.710370,***,1
1030,0.856344,0.854145,389.455469,***,2
1040,0.866123,0.864399,502.467672,***,3
1050,0.848081,0.846120,432.639212,***,3


###  Geographic stats

In [48]:
significantlu  ={}
for startingyear in range(960,1280,10):
    tempdf = coefdict[startingyear][6:35]
    significantlu[startingyear] = len(tempdf[tempdf.iloc[:,-1] != ''])
    print (tempdf)

Empty DataFrame
Columns: [960, 960mk]
Index: []
Empty DataFrame
Columns: [970, 970mk]
Index: []
Empty DataFrame
Columns: [980, 980mk]
Index: []
Empty DataFrame
Columns: [990, 990mk]
Index: []
Empty DataFrame
Columns: [1000, 1000mk]
Index: []
Empty DataFrame
Columns: [1010, 1010mk]
Index: []
Empty DataFrame
Columns: [1020, 1020mk]
Index: []
Empty DataFrame
Columns: [1030, 1030mk]
Index: []
Empty DataFrame
Columns: [1040, 1040mk]
Index: []
Empty DataFrame
Columns: [1050, 1050mk]
Index: []
Empty DataFrame
Columns: [1060, 1060mk]
Index: []
Empty DataFrame
Columns: [1070, 1070mk]
Index: []
Empty DataFrame
Columns: [1080, 1080mk]
Index: []
Empty DataFrame
Columns: [1090, 1090mk]
Index: []
Empty DataFrame
Columns: [1100, 1100mk]
Index: []
Empty DataFrame
Columns: [1110, 1110mk]
Index: []
Empty DataFrame
Columns: [1120, 1120mk]
Index: []
Empty DataFrame
Columns: [1130, 1130mk]
Index: []
Empty DataFrame
Columns: [1140, 1140mk]
Index: []
Empty DataFrame
Columns: [1150, 1150mk]
Index: []
Empty Da

In [49]:
dflist = []
for startingyear in range(960,1280,10):
    
    convertingdict = {}

    convertingdict['Longitude'] = coefdict[startingyear].iloc[4:6].iloc[0,0]
    convertingdict['Lonp'] = coefdict[startingyear].iloc[4:6].iloc[0,1]
    convertingdict['Latitude'] = coefdict[startingyear].iloc[4:6].iloc[1,0]
    convertingdict['Ltp'] = coefdict[startingyear].iloc[4:6].iloc[1,1]

    dflist.append(pd.DataFrame(convertingdict,index = [startingyear]))
lonlatdf = pd.concat(dflist)

In [50]:
geo = pd.concat([pd.DataFrame(significantlu, index = ['Number of Significant Lu']).T,lonlatdf],axis = 1)
geo

,Number of Significant Lu,Longitude,Lonp,Latitude,Ltp
960,0,-0.007477,,0.054763,
970,0,-0.146325,**,-0.145681,**
980,0,-0.082482,,-0.070995,
990,0,-0.027193,,0.042196,
1000,0,0.055638,,-0.042206,
1010,0,-0.071481,*,-0.057349,
1020,0,-0.012031,,-0.103000,***
1030,0,0.010928,,-0.125941,***
1040,0,-0.011952,,-0.103378,***
1050,0,0.051354,*,-0.127569,***


In [42]:
geo.to_csv('barebone_georesult.csv')

### Bloodline

In [ ]:
bloodroutes = ['entry_8','entry_59','entry_60','entry_62','entry_118']

In [ ]:
for startingyear in range(960,1280,10):
    print (coefdict[startingyear].T[bloodroutes])

In [ ]:
dflist = []

for startingyear in range(960,1280,10):
    tempdf = coefdict[startingyear].T[bloodroutes].T
    convertingdict ={}
    for i in range(len(bloodroutes)):
        convertingdict[bloodroutes[i]] = tempdf.iloc[i,0]
        convertingdict[bloodroutes[i]+'p'] = tempdf.iloc[i,1]

    dflist.append( pd.DataFrame(convertingdict, index = [startingyear]))

In [ ]:
pd.concat(dflist).to_csv('bloodresult.csv')

### Jinshi

In [ ]:
eliteroutes = ['entry_36','entry_165']

In [ ]:
dflist = []

for startingyear in range(960,1280,10):
    tempdf = coefdict[startingyear].T[eliteroutes].T
    convertingdict ={}
    for i in range(len(eliteroutes)):
        convertingdict[eliteroutes[i]] = tempdf.iloc[i,0]
        convertingdict[eliteroutes[i]+'p'] = tempdf.iloc[i,1]

    dflist.append( pd.DataFrame(convertingdict, index = [startingyear]))

In [ ]:
pd.concat(dflist).to_csv('eliteresult.csv')

### Network

In [35]:
network = ['status','nemesis']

In [36]:
dflist = []

for startingyear in range(960,1280,10):
    tempdf = coefdict[startingyear].T[network].T
    convertingdict ={}
    for i in range(len(network)):
        convertingdict[network[i]] = tempdf.iloc[i,0]
        convertingdict[network[i]+'p'] = tempdf.iloc[i,1]

    dflist.append( pd.DataFrame(convertingdict, index = [startingyear]))

In [43]:
pd.concat(dflist).to_csv('barebone_networkresult.csv')

# Random Forest

In [38]:
conninput = sqlite3.connect("edgelist.db")

In [39]:
def randomforestandreturn(year):
    
    # Read raw file
    raw = pd.read_csv('dataset'+str(year)+'.csv') 
    
    # Fill missing value: years, ages with mean
    for columnname in raw.iloc[:,7:15].columns:
        meanvalue = raw[raw[columnname] !=0][columnname].mean()
        raw[columnname] = raw[columnname].replace({0:meanvalue})
    
    
    dataset_X = pd.concat([raw.iloc[:,7],raw.iloc[:,9:11], raw.iloc[:,12:-4],raw.iloc[:,-2:]], axis = 1)
    dataset_y = raw.iloc[:,2]

    # Perform Grid-Search 
    gsc = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={
            'max_depth': range(3,20),
            'n_estimators': (10, 50, 100, 1000),
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

    grid_result = gsc.fit(dataset_X, dataset_y)
    best_params = grid_result.best_params_

    sel = SelectFromModel(RandomForestRegressor(max_depth=best_params["max_depth"],
                                n_estimators=best_params["n_estimators"],
                                random_state=False, verbose=False))
    sel.fit(dataset_X, dataset_y)

    rfr = RandomForestRegressor(max_depth=best_params["max_depth"],
                                n_estimators=best_params["n_estimators"],
                                random_state=False, verbose=False)
    # Perform K-Fold CV
    scores = cross_val_score(rfr, dataset_X, dataset_y, cv=10, scoring='neg_mean_absolute_error')
    predictions = cross_val_predict(rfr, dataset_X,dataset_y, cv=10)
    MSE = metrics.mean_squared_error(dataset_y, predictions)

    #    return predictions
    
    rfr.fit(dataset_X, dataset_y)
    
    return MSE, pd.DataFrame(rfr.feature_importances_, index = dataset_X.columns, columns = [year]).T

In [ ]:
connoutput =  sqlite3.connect("gini_finalR.db")

In [40]:
msedict = {}
ginidict = {}

In [41]:
for startingyear in range(960,1280,10):
    
    if startingyear in msedict:
        continue
    
    print('processing '+str(startingyear))
    
    mse, gini = randomforestandreturn(startingyear)

    #gini.to_sql('gini'+str(startingyear), con=connoutput, if_exists='replace')
    
    msedict[startingyear] = mse
    ginidict[startingyear] = gini

processing 960


C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


processing 970


KeyboardInterrupt: 

In [ ]:
msetable = pd.DataFrame(msedict, ['mse']).T
    

In [ ]:
msetable['sd'] = np.sqrt(msetable['mse'])

In [ ]:
msetable#.to_csv('newmse.csv')

In [ ]:
convertingdict ={}

for startingyear in range(960,1280,10):

    convertingdict[startingyear] = ginidict[startingyear].T.sort_values(by = startingyear, ascending = False).index[0:10]


In [ ]:
pd.DataFrame(convertingdict, index = range(1,11)).T.to_csv('giniresult.csv')

In [ ]:
range(960,1280,10)

In [ ]:
pd.DataFrame(msedict, index = ['mse']).T.to_csv('mse.csv')

In [ ]:
ginis = {}
for year in range(960,1280,10):
    ginis[year]= pd.read_sql_query("SELECT * FROM gini"+str(year), 
                         sqlite3.connect('gini_finalR.db')).drop_duplicates()

In [ ]:
dflis = []
for year in range(960,1280,10):

    tempdf = ginis[year].T.sort_values(by = 0, ascending = False).iloc[1:11].reset_index()
    tempdf.columns = [str(year)+'+factor',str(year)+'+gini']
    dflis.append(tempdf)

In [ ]:
pd.concat(dflis, axis =1).to_csv('ginipresent10.csv')

In [ ]:
t2df.loc[960][0] = str(tempdf.index[0])
t2df

In [ ]:
pd.DataFrame(tempdict,index = ['960'])

### Coefficient

In [ ]:
dflist = []
for year in coefdict:
    dflist.append(coefdict[year])

In [ ]:
coeffull = pd.concat(dflist, axis = 1)

In [ ]:
coeffull.to_csv('coefficient.csv')

In [ ]:
testdf

In [ ]:
pearsondict = {}

for year in range(960,1280,10):

    raw = pd.read_csv('dataset'+str(year)+'.csv') 
    
    pearsondict[year]=stats.pearsonr(raw.iloc[:,-2],raw.iloc[:,-1])

In [ ]:
pd.DataFrame(pearsondict, index = ['corr','p-value']).T.to_csv('nemesis-status-correlation.csv')

In [ ]:
for year in range(960,1280,10):

    raw = pd.read_csv('dataset'+str(year)+'.csv') 
    
    print(raw.iloc[:,-2])
    print (raw.iloc[:,-1])